In [ ]:
!pip3 install datasets evaluate sentencepiece lxml wandb accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
!wandb login 094590d0aa8813c0cc044d53c48dbf393da80d96

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import pandas as pd
import numpy as np
import os
from transformers import  (AutoModel,AutoModelForSequenceClassification,
                           DataCollatorWithPadding,AutoTokenizer,TrainingArguments,Trainer)
from datasets import Dataset,DatasetDict
import evaluate
accuracy = evaluate.load("accuracy")
import torch
from sklearn.metrics import accuracy_score
from datasets import load_metric
import sentencepiece

def compute_metrics(eval_pred):
    metrics = ["accuracy", "recall", "precision", "f1"] #List of metrics to return
    metric={}
    for met in metrics:
       metric[met] = load_metric(met)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric_res={}
    for met in metrics:
       metric_res[met]=metric[met].compute(predictions=predictions, references=labels)[met]
    return metric_res


def csv_to_df(path: str = None,question_type:int=3, sample: int = 0) -> pd.DataFrame:
    """

    :param path: "csv file path
    :param question_type: for q1 ->1, for q2->2, for q1+q2->3
    :param sample: 0 is full dataset, anything else is number of rows
    :return: pandas dataframe
    """
    df = pd.read_csv(path, encoding="ISO-8859-1")
    print(question_type, sample)
    if sample != 0:
        df = df.sample(n=sample, replace='False')


    df.loc[df['outcome_class'] == 't', 'outcome_class'] = 1
    df.loc[df['outcome_class'] == 'd', 'outcome_class'] = 0
    if question_type == 1:
        df['q'] = df['q1'].apply(lambda x: x.replace('\n', ''))
    elif question_type ==2:
        df['q'] = df['q2'].apply(lambda x: x.replace('\n', ''))
    elif question_type == 3:
        df['q1'] = df['q1'].apply(lambda x: x.replace('\n', ''))
        df['q2'] = df['q2'].apply(lambda x: x.replace('\n', ''))
        df['q'] = df['q1'] + df['q2']

    df = df.rename(columns={'q': 'sent', 'outcome_class': 'label'})
    df = df[['sent', 'label']]
    return df
def tokenize_function(examples: Dataset) -> Dataset:
    model_inputs = tokenizer(
        examples["sent"],
        truncation=True,
        padding="max_length",  # Add this line to pad to the maximum length
        max_length=512,  # Set the maximum length to match your model's limit
    )
    return model_inputs

seed = 42
np.random.seed(seed)
data_path = 'data/sign_events_data_statements.csv'
intent_df = csv_to_df(path=data_path,
                      question_type= 3,
                      sample = 0)

dataset = Dataset.from_pandas(intent_df)
train_testvalid = dataset.train_test_split(test_size= 0.1, shuffle=True)
valid_test = train_testvalid["test"].train_test_split(test_size=0.5,shuffle=True)
dataset_splitted = DatasetDict({
    'train': train_testvalid["train"],
    'valid': valid_test["train"],
    'test': valid_test["test"]
})


3 0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
import os

checkpoint = "microsoft/Multilingual-MiniLM-L12-H384"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

path = os.path.join("./models/pretrained",checkpoint)
tokenizer.save_pretrained(path)
model.save_pretrained(path)

tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
checkpoint = "microsoft/Multilingual-MiniLM-L12-H384"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Print the maximum token limit
max_token_limit = tokenizer.model_max_length
print(f"Maximum Token Limit: {max_token_limit}")

Maximum Token Limit: 1000000000000000019884624838656


In [ ]:
checkpoint = "microsoft/Multilingual-MiniLM-L12-H384"
path = os.path.join("./models/pretrained",checkpoint)
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)
id2label = {0: "F", 1: "T"}
label2id = {"F": 0, "T": 1}

model = AutoModelForSequenceClassification.from_pretrained(
path, num_labels=2, id2label=id2label, label2id=label2id
                                   )
tokenized_dataset = dataset_splitted.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

for param in model.bert.encoder.layer[:8].parameters():  # Freeze the first 2 layers of the encoder
    param.requires_grad = False

Map:   0%|          | 0/1476 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

In [ ]:
import os
import shutil

# Specify the directory you want to remove
directory_to_remove = "my_awesome_model"

# Check if the directory exists before attempting to remove it
if os.path.exists(directory_to_remove):
    # Use shutil.rmtree to remove the directory and its contents
    shutil.rmtree(directory_to_remove)
    print(f"The directory '{directory_to_remove}' has been removed.")
else:
    print(f"The directory '{directory_to_remove}' does not exist.")


The directory 'my_awesome_model' has been removed.


In [ ]:
num_epochs = 30
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_d evice_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_cpu= False
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()



wandb: Currently logged in as: maatvo. Use `wandb login --relogin` to force relogin


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,No log,0.683464,0.573171,0.027778,1.000000,0.054054
2,No log,0.612300,0.707317,0.777778,0.636364,0.700000
3,No log,0.598237,0.695122,0.638889,0.657143,0.647887
4,No log,0.761913,0.560976,0.805556,0.500000,0.617021
5,No log,0.699741,0.646341,0.638889,0.589744,0.613333
6,0.562500,0.757800,0.646341,0.694444,0.581395,0.632911
7,0.562500,0.811655,0.609756,0.694444,0.543478,0.609756
8,0.562500,0.737554,0.670732,0.611111,0.628571,0.619718
9,0.562500,0.727558,0.719512,0.611111,0.709677,0.656716
10,0.562500,0.927738,0.560976,0.666667,0.500000,0.571429


<ipython-input-1-63e75cb636ba>:18: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric[met] = load_metric(met)


TrainOutput(global_step=2790, training_loss=0.2866446026764463, metrics={'train_runtime': 2015.2365, 'train_samples_per_second': 21.973, 'train_steps_per_second': 1.384, 'total_flos': 2916844489359360.0, 'train_loss': 0.2866446026764463, 'epoch': 30.0})

In [ ]:
# Prediction
predictions = []
labels = []
for example in tokenized_dataset["test"]:
    torch.cuda.empty_cache()

    # Tokenize input
    inputs = tokenizer(example['sent'], return_tensors='pt')

    inputs.to('cuda')
    outputs = model(**inputs)

    # Get predictions and labels
    prediction = torch.argmax(outputs.logits).item()
    label = example['label']  # Replace 'label' with the actual key in your dataset

    predictions.append(prediction)
    labels.append(label)

# Calculate accuracy
accuracy = accuracy_score(labels, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 67.07%
